In [1]:
// Para trabajar con los DataFrame
import org.apache.spark.sql.DataFrame
// Para trabajar con SparkSQL
import org.apache.spark.sql.SparkSession

Intitializing Scala interpreter ...

Spark Web UI available at http://10.0.2.15:4040
SparkContext available as 'sc' (version = 2.4.6, master = local[*], app id = local-1616012424623)
SparkSession available as 'spark'


import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.Dataset
import spark.implicits._
import org.apache.spark.sql.SparkSession


In [2]:
// Se instancia SparkSQL
val spark = SparkSession.builder.appName("miApp").master("local").getOrCreate()

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@3113804d


# Ejercicio 1
Haciendo uso del archivo de twitter, contestar a las sisguientes preguntas:
* ¿Cuantos tweets contiene el archivo?
* ¿Cuantos tweets estan en ingles?
* ¿De cuantos usuarios distintos hay algun tweet en el archivo?

In [4]:
// Leemos los datos del archivo como JSON
val tweetsJSON = spark.read.json("/home/vmuser/curso/large-17-json")

tweetsJSON: org.apache.spark.sql.DataFrame = [contributors: string, coordinates: struct<coordinates: array<double>, type: string> ... 22 more fields]


In [5]:
// Mostramos la estructura del archivo
tweetsJSON.printSchema()

root
 |-- contributors: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- display_url: string (nullable = true)
 |    |    |    |-- expanded_url: string (nullable = true)
 |    |    |    |-- id: long (nullable = true)
 |    |    |    |-- id_str: string (nullable = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true

In [10]:
// Contamos los registros del archivo
val numTweets = tweetsJSON.count()
println("Hay "+numTweets+" tweets en el archivo")

Hay 515361 tweets en el archivo


numTweets: Long = 515361


In [11]:
// Hacemos un filtro y contamos el resultado
val tweetsIngles = tweetsJSON.filter($"lang"==="en").count()
println("Hay "+tweetsIngles+" tweets en ingles")

Hay 356218 tweets en ingles


tweetsIngles: Long = 356218


In [21]:
// Seleccionamos los registros de la columna indicada
val usuario = tweetsJSON.select("user.screen_name")
// Contamos los registros seleccionados diferentes entre si
val distintos = usuario.distinct().count()
println("Hay "+distintos+" usuarios en el archivo")

Hay 352736 usuarios en el archivo


usuario: org.apache.spark.sql.DataFrame = [screen_name: string]
distintos: Long = 352736


# Ejercicio 2
Definir una clase Tweet con la siguiente estructur:
* Tweet
    * Fecha creacion
    * Nombre usuario
    * Numero seguidores
    * Texto
    
Luego, convertir el DataFrame del Ejecicio 1 en un DataSet, y responder:
* ¿Quien es el usuario con mayor número de seguidores?

In [54]:
// Para trabajar con los DataSet
import org.apache.spark.sql.Dataset
import spark.implicits._

import org.apache.spark.sql.Dataset
import spark.implicits._


In [55]:
// Definimos la clase Tweet
case class Tweet(Fecha_creacion: Option[String], Nombre_usuario: Option[String],
                Numero_seguidores: Option[Long], Texto: Option[String])

defined class Tweet


In [56]:
// Seleccionamos los atributos del archivo
val tweetsDF = tweetsJSON.select("created_at", "user.screen_name",
                                "user.followers_count", "text")
// Se muestran como una tabla
tweetsDF.show()

+--------------------+---------------+---------------+--------------------+
|          created_at|    screen_name|followers_count|                text|
+--------------------+---------------+---------------+--------------------+
|Fri Dec 21 22:53:...|   washexaminer|           1995|Obama meets Reid ...|
|Fri Dec 21 22:53:...|narrychristmas_|            269|@halluzaynating o...|
|Fri Dec 21 22:53:...|  nicolehoran08|            120|@hii1d Awww. Poor...|
|Fri Dec 21 22:53:...|     Odemanjr15|             61|All you need in a...|
|Fri Dec 21 22:53:...|       colovion|            490|RT @PatDollard: W...|
|Fri Dec 21 22:53:...|Coolan_n_Foolan|            153|Crankin to this F...|
|Fri Dec 21 22:53:...|     YoSoyDalia|           2081|RT @InjusticeFact...|
|Fri Dec 21 22:53:...|   KinderdijkNL|             42|Prio 2 TS500 Bran...|
|Fri Dec 21 22:53:...|    akmalarif90|            545|RT @BBCWorld: Bar...|
|Fri Dec 21 22:53:...|   organicsblue|              1|Amazon Kindle Pap...|
|Fri Dec 21 

tweetsDF: org.apache.spark.sql.DataFrame = [created_at: string, screen_name: string ... 2 more fields]


In [57]:
// Le damos nombre a las columnas del dataframe anterior,
// y lo convertimos a dataset al otorgarle la clase Tweet
val tweetsDS = tweetsDF.toDF("Fecha_creacion", "Nombre_usuario",
                            "Numero_seguidores", "Texto").as[Tweet]
// Se sigue mostrando como una tabla
tweetsDS.show()

+--------------------+---------------+-----------------+--------------------+
|      Fecha_creacion| Nombre_usuario|Numero_seguidores|               Texto|
+--------------------+---------------+-----------------+--------------------+
|Fri Dec 21 22:53:...|   washexaminer|             1995|Obama meets Reid ...|
|Fri Dec 21 22:53:...|narrychristmas_|              269|@halluzaynating o...|
|Fri Dec 21 22:53:...|  nicolehoran08|              120|@hii1d Awww. Poor...|
|Fri Dec 21 22:53:...|     Odemanjr15|               61|All you need in a...|
|Fri Dec 21 22:53:...|       colovion|              490|RT @PatDollard: W...|
|Fri Dec 21 22:53:...|Coolan_n_Foolan|              153|Crankin to this F...|
|Fri Dec 21 22:53:...|     YoSoyDalia|             2081|RT @InjusticeFact...|
|Fri Dec 21 22:53:...|   KinderdijkNL|               42|Prio 2 TS500 Bran...|
|Fri Dec 21 22:53:...|    akmalarif90|              545|RT @BBCWorld: Bar...|
|Fri Dec 21 22:53:...|   organicsblue|                1|Amazon K

tweetsDS: org.apache.spark.sql.Dataset[Tweet] = [Fecha_creacion: string, Nombre_usuario: string ... 2 more fields]


In [58]:
// Nos otorga funciones para SparkSQL
import org.apache.spark.sql.functions._
// Ordenamos de manera descendente segun la columna "Numero_Seguidores"
val ordenados = tweetsDS.orderBy(desc("Numero_seguidores"))
// Se muestra el resultado como una tabla
ordenados.show()

+--------------------+--------------+-----------------+--------------------+
|      Fecha_creacion|Nombre_usuario|Numero_seguidores|               Texto|
+--------------------+--------------+-----------------+--------------------+
|Sat Dec 22 00:38:...|   BarackObama|         24829148|RT @whitehouse: "...|
|Sat Dec 22 00:03:...|        cnnbrk|          9726909|Obama outlines a ...|
|Sat Dec 22 00:56:...|  RyanSeacrest|          8387001|RT @BAHjournalist...|
|Sat Dec 22 01:37:...|       nytimes|          6903302|$60.4 Billion Hur...|
|Sat Dec 22 02:55:...|           CNN|          6861562|Opinion: What Oba...|
|Sat Dec 22 00:23:...|           CNN|          6860745|Blizzard causes c...|
|Sat Dec 22 00:07:...|           CNN|          6860700|RT @OutFrontCNN: ...|
|Fri Dec 21 22:57:...|           CNN|          6860411|RT @PoliticalTick...|
|Sat Dec 22 01:44:...|           NBA|          6409870|Thad Young has 16...|
|Sat Dec 22 01:03:...|           NBA|          6409728|Zaza Pachulia bea...|

import org.apache.spark.sql.functions._
ordenados: org.apache.spark.sql.Dataset[Tweet] = [Fecha_creacion: string, Nombre_usuario: string ... 2 more fields]


In [69]:
// Se imprime el primer registro(solo la columna "Nombre_usuario")
// dataset anterior
val mas_seguidores = ordenados.select("Nombre_usuario").limit(1).show()

+--------------+
|Nombre_usuario|
+--------------+
|   BarackObama|
+--------------+



mas_seguidores: Unit = ()


# Ejercicio 3
Haciendo uso del DataSet del ejericio 2:
* Aplicarle persistencia solo en disco
* ¿Cuantos tweets contienen la palabra "data"?
* Volcar el dataset en un archivo JSON

In [70]:
// Ofrece herramientas para Persistencia
import org.apache.spark.storage.StorageLevel

import org.apache.spark.storage.StorageLevel


In [71]:
// Se le aplica persistencia solo en disco al dataset
tweetsDS.persist(StorageLevel.DISK_ONLY)

res44: tweetsDS.type = [Fecha_creacion: string, Nombre_usuario: string ... 2 more fields]


In [74]:
// Se filtran los registros que en su columna Texto contengan
// la palabra "data" y se cuentan estos registros
val conData = tweetsDS.filter($"Texto".like("% data %")).count()

conData: Long = 161


In [79]:
// Seleccionamos la columna Texto de los registros
// Para cada valor de estos registros, los convertimos a strings y
// los dividimos por palabras (en donde haya " ") y se nos retorna un array
val conData2 = tweetsDS.select("Texto").map(p => p.toString().split(" "))
// Se filtran los registros contando solo los que contengan el
// string "data"
conData2.filter(_.contains("data")).count()

conData2: org.apache.spark.sql.Dataset[Array[String]] = [value: array<string>]
res48: Long = 161


In [82]:
// Creamos un archivo que contiene el dataset "tweetsDS"
tweetsDS.write.json("/home/vmuser/curso/datasetTweets")